## Imports

In [1]:
# import NALU files
# import sys
# sys.path.insert(0, 'C:/Users/Ammar/Documents/CV LAB internship/Sproj/NALU_files')
# from nalu import NALU
# from nac import NAC

# import Keras items
from keras.models import Model
from keras.layers import Dense,Lambda,Concatenate,Input
from keras import backend as K
from keras.utils import to_categorical,plot_model
from keras.initializers import constant

# import other libs
import numpy as np
from itertools import combinations_with_replacement,product
# from sklearn.model_selection import train_test_split

from tensorflow import floor as tf_floor

Using TensorFlow backend.


## Parameters

In [2]:
model_name = 'adder_0-9'
EPOCHS = 2000
BATCH_SIZE = 4
range_min = 0
range_max = 9

## Generate Dataset

In [3]:
x = np.array(list(product(range(10),repeat = 2)))
y = np.empty((100,3))
y_sum = x[:,0] + x[:,1]
y[:,0] = np.floor(y_sum / 10)
y[:,1] = y_sum % 10
y[:,2] = y_sum
# y = to_categorical(y,num_classes=19)
print('\t',y.shape)

	 (100, 3)


## Model

In [13]:
def net(inputs):
    input_data = Input(shape = inputs)
    addition = Dense(1,kernel_initializer='ones',use_bias=False)(input_data)
    
    carry = Dense(1,use_bias=False,kernel_initializer=constant(value=0.1))(addition)
    carry = Lambda(lambda x: tf_floor(x))(carry)
    
    ones_digit = Dense(1,use_bias=False,kernel_initializer=constant(value=10))(carry)
    temp = Concatenate()([ones_digit,addition])
    ones_digit = Dense(1,use_bias=False)(temp)
    
    out = Concatenate()([carry,ones_digit,addition])
    return Model(inputs = input_data,outputs = out)

+ #### Model Initilization And Details

In [18]:
model = net((2,))
model.layers[6].set_weights([np.array(([-1],[1]))])
model.trainable = False
model.compile(loss = 'MSE',optimizer = 'Adam',metrics = ['accuracy'])
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_12 (InputLayer)           (None, 2)            0                                            
__________________________________________________________________________________________________
dense_45 (Dense)                (None, 1)            2           input_12[0][0]                   
__________________________________________________________________________________________________
dense_46 (Dense)                (None, 1)            1           dense_45[0][0]                   
__________________________________________________________________________________________________
lambda_12 (Lambda)              (None, 1)            0           dense_46[0][0]                   
__________________________________________________________________________________________________
dense_47 (

+ #### Fit Model

In [41]:
fitted = model.fit(x, y, validation_data = (x,y),
          batch_size=BATCH_SIZE, epochs=EPOCHS,
          verbose=1)

ValueError: Error when checking target: expected lambda_9 to have 1 dimensions, but got array with shape (100, 1)

+ #### Test Model

In [19]:
out = model.predict(x)
for i in range(100):
    print ('\t',x[i,:],' = ', y[i])
    print('\t predicted: ',out[i])

	 [0 0]  =  [ 0.  0.  0.]
	 predicted:  [ 0.  0.  0.]
	 [0 1]  =  [ 0.  1.  1.]
	 predicted:  [ 0.  1.  1.]
	 [0 2]  =  [ 0.  2.  2.]
	 predicted:  [ 0.  2.  2.]
	 [0 3]  =  [ 0.  3.  3.]
	 predicted:  [ 0.  3.  3.]
	 [0 4]  =  [ 0.  4.  4.]
	 predicted:  [ 0.  4.  4.]
	 [0 5]  =  [ 0.  5.  5.]
	 predicted:  [ 0.  5.  5.]
	 [0 6]  =  [ 0.  6.  6.]
	 predicted:  [ 0.  6.  6.]
	 [0 7]  =  [ 0.  7.  7.]
	 predicted:  [ 0.  7.  7.]
	 [0 8]  =  [ 0.  8.  8.]
	 predicted:  [ 0.  8.  8.]
	 [0 9]  =  [ 0.  9.  9.]
	 predicted:  [ 0.  9.  9.]
	 [1 0]  =  [ 0.  1.  1.]
	 predicted:  [ 0.  1.  1.]
	 [1 1]  =  [ 0.  2.  2.]
	 predicted:  [ 0.  2.  2.]
	 [1 2]  =  [ 0.  3.  3.]
	 predicted:  [ 0.  3.  3.]
	 [1 3]  =  [ 0.  4.  4.]
	 predicted:  [ 0.  4.  4.]
	 [1 4]  =  [ 0.  5.  5.]
	 predicted:  [ 0.  5.  5.]
	 [1 5]  =  [ 0.  6.  6.]
	 predicted:  [ 0.  6.  6.]
	 [1 6]  =  [ 0.  7.  7.]
	 predicted:  [ 0.  7.  7.]
	 [1 7]  =  [ 0.  8.  8.]
	 predicted:  [ 0.  8.  8.]
	 [1 8]  =  [ 0.  9.  9.]
	 

+ #### Save Model

In [14]:
model.save_weights('adder_0-9_classifier.h5')

In [151]:
plot_model(model,to_file = 'mul_0-9.png',show_shapes=True)